In [4]:
# Step1 : Install External modules
#----------------------------------------------------------------------------
"""
!python boto_client.py
!python llm_basemodel.py
!python vector_embedding.py
!python athena_execution.py
!python openSearchVCEmbedding.py
!python openSearchVCEmbedding.py
"""
#-----------------------------------------------------------------------------

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/langchain/embeddings/__init__.py:29: LangChainDeprecationWarning: Importing embeddings from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.embeddings import BedrockEmbeddings`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/langchain/document_loaders/__init__.py:36: LangChainDeprecationWarning: Importing document loaders from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.document_loaders import JSONLoader`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/py

In [6]:
!pip3 install boto3==1.34.8
!pip3 install jq
!pip3 install opensearch-py

In [7]:
# Step2 : Import External Modules 
#---------------------------------------------------------------------------------------
import boto3
from botocore.config import Config
from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings
from openSearchVCEmbedding import EmbeddingBedrockOpenSearch
from boto_client import Clientmodules
from llm_basemodel import LanguageModel
from athena_execution import AthenaQueryExecute
import logging 
import json
import os,sys
import re
sys.path.append("/home/ec2-user/SageMaker/llm_bedrock_v0/")
import time
import pandas as pd
import io
#--------------------------------------------------------------------------------------

In [8]:
# set the logger 
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [9]:
# Step3 : Checking access to Bedrock 
# To ensure sagemaker can access Bedrock Models 
#----------------------------------------------------------------------------------
session = boto3.session.Session()
bedrock_client = session.client('bedrock')
#print(bedrock_client.list_foundation_models()['modelSummaries'][0])

2024-04-03 12:08:47,782,credentials,MainProcess,INFO,Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [10]:
# Step4 : Invoking AthenaClient to check the syntax of query genrated by model and tp get the 
#         response of recommended query
rqstath=AthenaQueryExecute()

2024-04-03 12:08:54,554,credentials,MainProcess,INFO,Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
athena client created 
2024-04-03 12:08:54,808,boto_client,MainProcess,INFO,athena client created 
2024-04-03 12:08:54,847,credentials,MainProcess,INFO,Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
s3 client created !!
2024-04-03 12:08:54,958,boto_client,MainProcess,INFO,s3 client created !!


In [11]:
# Step5 :  Invoking OpenSearch client to add index in OpenSearch 
ebropen=EmbeddingBedrockOpenSearch()

2024-04-03 12:08:55,003,credentials,MainProcess,INFO,Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
bedrock runtime client created 
2024-04-03 12:08:55,056,boto_client,MainProcess,INFO,bedrock runtime client created 


In [ ]:
#Step6 : Add index to OpenSearch domain using schema of the database 
#index_name = 'llm_vector_db_metadata_indx1'
#file_name = 'imdb_schema.jsonl'
#ebropen.add_documnets(index_name,file_name)

In [12]:
# Step7: To create a class to get vector embedding and to generate sql 
class RequestQueryBedrock:
    def __init__(self):
        # self.model_id = "anthropic.claude-v2"
        self.bedrock_client = Clientmodules.createBedrockRuntimeClient()
        self.language_model = LanguageModel(self.bedrock_client)
        self.llm = self.language_model.llm
    def getOpenSearchEmbedding(self,index_name,user_query):
        vcindxdoc=ebropen.getDocumentfromIndex(index_name=index_name)
        documnet=ebropen.getSimilaritySearch(user_query,vcindxdoc)
        return ebropen.format_metadata(documnet)
        
    def generate_sql(self,prompt, max_attempt=4) ->str:
            """
            Generate and Validate SQL query.

            Args:
            - prompt (str): Prompt is user input and metadata from Rag to generating SQL.
            - max_attempt (int): Maximum number of attempts correct the syntax SQL.

            Returns:
            - string: Sql query is returned .
            """
            attempt = 0
            error_messages = []
            prompts = [prompt]

            while attempt < max_attempt:
                logger.info(f'Sql Generation attempt Count: {attempt+1}')
                try:
                    logger.info(f'we are in Try block to generate the sql and count is :{attempt+1}')
                    generated_sql = self.llm.predict(prompt)
                    query_str = generated_sql.split("```")[1]
                    query_str = " ".join(query_str.split("\n")).strip()
                    sql_query = query_str[3:] if query_str.startswith("sql") else query_str
                    # return sql_query
                    syntaxcheckmsg=rqstath.syntax_checker(sql_query)
                    if syntaxcheckmsg=='Passed':
                        logger.info(f'syntax checked for query passed in attempt number :{attempt+1}')
                        return sql_query
                    else:
                        prompt = f"""{prompt}
                        This is syntax error: {syntaxcheckmsg}. 
                        To correct this, please generate an alternative SQL query which will correct the syntax error.
                        The updated query should take care of all the syntax issues encountered.
                        Follow the instructions mentioned above to remediate the error. 
                        Update the below SQL query to resolve the issue:
                        {sqlgenerated}
                        Make sure the updated SQL query aligns with the requirements provided in the initial question."""
                        prompts.append(prompt)
                        attempt += 1
                except Exception as e:
                    logger.error('FAILED')
                    msg = str(e)
                    error_messages.append(msg)
                    attempt += 1
            return sql_query
print("Completed...")

Completed...


In [13]:
#Step8 : create the object of above defined class to generate sql 
index_name = 'llm_vector_db_metadata_indx1'
rqst=RequestQueryBedrock()

2024-04-03 12:09:11,175,credentials,MainProcess,INFO,Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
bedrock runtime client created 
2024-04-03 12:09:11,238,boto_client,MainProcess,INFO,bedrock runtime client created 


In [46]:
# Step9 : Create a function to generate user query along with prompt and vector_search 
def userinput(user_query):
    logger.info(f'Searching metadata from vector store')
    # vector_search_match=rqst.getEmbeddding(user_query)
    vector_search_match=rqst.getOpenSearchEmbedding(index_name,user_query)
    # print(vector_search_match)
    details="""It is important that the SQL query complies with Athena syntax. 
               During join if column name are same please use alias ex llm.customer_id in select statement. 
               Generate an SQL query that retrieves data from multiple tables efficiently using appropriate joins and conditions
               It is also important to respect the type of columns: if a column is string, the value should be enclosed in 
               quotes. While concatenating a non string column, make sure cast the column to string. 
               Efficiently change the data type during comparison like int to double or vice versa.
               Example :
               For int column comparing to float , please cast the int input
               For date columns comparing to string , please cast the string input.
               "Write an SQL query that identifies and removes duplicate records from a table efficiently, considering both time and cost factors"""
    final_question = "\n\nHuman:"+ details +"\n Following  is the database metadata for your reference: "+ vector_search_match +"\n Considering above instructions and taking reference of schema Answer the below query:"+ user_query+ "\n\nAssistant:"
    print(final_question)
    answer = rqst.generate_sql(final_question)
    return answer

In [47]:
# Step 10 : Set user query
#user_query='show me all the titles in US region'
#user_query = 'show me all the regions in which title(s) are availble'
user_query = 'give me all the original title names and their rating, note : rating should not be less than 7.5'
#user_query ='show me the distinct top 10 ten title name and the votes they have received'
#user_query = 'Show me all the title and details in US region whose rating is more than 9.5.'
#user_query =  'Great Response! Now show me all the original type titles having ratings more than 7.5 and not in the US region.'

In [48]:
# Step 11: Result-------------------------------
# Generate Output
querygenerated=userinput(user_query)
#print(querygenerated)

Searching metadata from vector store
2024-04-03 12:38:53,589,2369146729,MainProcess,INFO,Searching metadata from vector store
2024-04-03 12:38:53,795,base,MainProcess,INFO,POST https://search-vectoddbforntosql-eiooyl4xmfiuptjriaitwnymy4.us-east-1.es.amazonaws.com:443/llm_vector_db_metadata_indx1/_search [status:200 request:0.053s]
Sql Generation attempt Count: 1
2024-04-03 12:38:53,797,569136725,MainProcess,INFO,Sql Generation attempt Count: 1
we are in Try block to generate the sql and count is :1
2024-04-03 12:38:53,803,569136725,MainProcess,INFO,we are in Try block to generate the sql and count is :1




Human:It is important that the SQL query complies with Athena syntax. 
               During join if column name are same please use alias ex llm.customer_id in select statement. 
               Generate an SQL query that retrieves data from multiple tables efficiently using appropriate joins and conditions
               It is also important to respect the type of columns: if a column is string, the value should be enclosed in 
               quotes. While concatenating a non string column, make sure cast the column to string. 
               Efficiently change the data type during comparison like int to double or vice versa.
               Example :
               For int column comparing to float , please cast the int input
               For date columns comparing to string , please cast the string input.
               "Write an SQL query that identifies and removes duplicate records from a table efficiently, considering both time and cost factors
 Following  is the database met

syntax checked for query passed in attempt number :1
2024-04-03 12:39:03,771,569136725,MainProcess,INFO,syntax checked for query passed in attempt number :1


Status : {'State': 'SUCCEEDED', 'SubmissionDateTime': datetime.datetime(2024, 4, 3, 12, 39, 0, 713000, tzinfo=tzlocal()), 'CompletionDateTime': datetime.datetime(2024, 4, 3, 12, 39, 1, 435000, tzinfo=tzlocal())}


In [49]:
# Step 12: Print Query result 
import pprint
my_printer = pprint.PrettyPrinter()
my_printer.pprint(querygenerated)

(' SELECT t.title, tr.averageRating  FROM imdb_stg.title t INNER JOIN '
 'imdb_stg.title_ratings tr ON t.titleId = tr.tconst WHERE t.isOriginalTitle = '
 "'1' AND CAST(tr.averageRating AS double) >= 7.5")


In [50]:
#Step 13 : Execute the above generated  query 
print("In last block........")
QueryOutput=rqstath.execute_query(querygenerated)
print(QueryOutput)
print("completed........")

In last block........


checking for file :athena_output/b674c8b2-4b0f-4c2f-97d6-94e6ef7f5cbb.csv
2024-04-03 12:41:35,037,athena_execution,MainProcess,INFO,checking for file :athena_output/b674c8b2-4b0f-4c2f-97d6-94e6ef7f5cbb.csv


Calling download fine with params /tmp/athena_output/b674c8b2-4b0f-4c2f-97d6-94e6ef7f5cbb.csv, {'OutputLocation': 's3://tesawsbucketdesign1/athena_output'}
                                   title  averageRating
0                          Purr Cat Cafe            7.5
1                  The Loss of the Perch            7.9
2                 The Triton's Christmas            8.3
3                   Rebel Without a Pawn            8.0
4       Tower of Babel: The New Evidence            8.0
...                                  ...            ...
560175                 The Eternal Derby            8.1
560176   Bare Feet with Mickela Mallozzi            7.5
560177                  Full Steam Ahead            8.4
560178                    O último Andar            7.7
560179                Bucky and the Howl            7.5

[560180 rows x 2 columns]
completed........
